# Épico 2.1: Importação e Extração de Dados de Extratos de Folha de Pagamento

Este notebook documenta e prototipa as funcionalidades relacionadas à importação e extração de dados de extratos de folha de pagamento, com foco no uso do Document AI para processar PDFs e no armazenamento dos dados extraídos no BigQuery.

# Módulo 2: Controle Mensal da Folha Inteligente

Este notebook documenta e prototipa as funcionalidades do Módulo 2, incluindo a importação de extratos de folha de pagamento em formato PDF, processamento assíncrono usando o Document AI e armazenamento no BigQuery.

# Processamento Assíncrono do PDF

Nesta seção, será demonstrado como o sistema realiza o processamento assíncrono de arquivos PDF utilizando o Document AI e integra os resultados ao BigQuery.

In [ ]:
import requests
import json

# Configurações
API_BASE_URL = "http://localhost:8000"
CLIENT_ID = "12345"
PDF_FILE_PATH = "sample_extrato.pdf"

# Enviar PDF para processamento
with open(PDF_FILE_PATH, 'rb') as pdf_file:
    response = requests.post(
        f"{API_BASE_URL}/api/v1/clientes/{CLIENT_ID}/folhas/importar-pdf-async",
        files={"file": pdf_file}
    )

if response.status_code == 200:
    job_id = response.json().get("job_id")
    print(f"Job iniciado com sucesso. ID do Job: {job_id}")
else:
    print(f"Erro ao iniciar o job: {response.text}")

# Configuração do Ambiente

Nesta seção, configuramos as bibliotecas necessárias, autenticação com Google Cloud e inicializamos variáveis globais.

In [ ]:
# Importar bibliotecas necessárias
import os
import pandas as pd
from google.cloud import storage
from google.oauth2 import service_account

# Configurar autenticação com Google Cloud
credentials = service_account.Credentials.from_service_account_file(
    'path/to/your-service-account-key.json'
)
client = storage.Client(credentials=credentials)

# Inicializar variáveis globais
BUCKET_NAME = 'seu-bucket-folhas-clientes'

# Upload de Arquivo PDF

Nesta seção, implementamos a interface para upload de arquivos PDF usando Streamlit e salvamos os arquivos no Google Cloud Storage.

In [ ]:
import streamlit as st

# Interface para upload de arquivos PDF
st.title("Upload de Arquivo PDF")

uploaded_file = st.file_uploader("Selecione o arquivo PDF:", type=["pdf"])

if uploaded_file is not None:
    st.write("Arquivo carregado com sucesso!")
    # Salvar o arquivo no Google Cloud Storage
    bucket = client.bucket(BUCKET_NAME)
    blob = bucket.blob(f"uploads/{uploaded_file.name}")
    blob.upload_from_file(uploaded_file)
    st.success(f"Arquivo {uploaded_file.name} salvo no bucket {BUCKET_NAME}.")

# Mapeamento e Validação de Dados

Nesta seção, será demonstrado como os dados extraídos do PDF são mapeados para a estrutura interna do sistema e validados antes de serem armazenados no BigQuery.

In [ ]:
import pandas as pd

# Simulação de dados extraídos do Document AI
data_extracted = [
    {"funcionario_nome": "João Silva", "cpf": "123.456.789-00", "rubrica": "Salário", "valor": 5000.00},
    {"funcionario_nome": "Maria Oliveira", "cpf": "987.654.321-00", "rubrica": "Vale Transporte", "valor": -150.00},
]

# Criar DataFrame para validação
df = pd.DataFrame(data_extracted)

# Validação de dados
def validate_data(row):
    errors = []
    if not row['cpf']:
        errors.append("CPF ausente")
    if row['valor'] <= 0:
        errors.append("Valor inválido")
    return errors

df['erros'] = df.apply(validate_data, axis=1)

# Exibir resultados de validação
print("Dados validados:")
print(df)

# Armazenamento no BigQuery

Nesta seção, será demonstrado como os dados validados são armazenados no BigQuery, utilizando a biblioteca `google-cloud-bigquery` para interagir com o serviço.

In [ ]:
from google.cloud import bigquery

# Configurar cliente do BigQuery
client = bigquery.Client()

def insert_into_bigquery(dataset_id, table_id, rows_to_insert):
    table_ref = client.dataset(dataset_id).table(table_id)
    errors = client.insert_rows_json(table_ref, rows_to_insert)
    if errors:
        print(f"Erros ao inserir no BigQuery: {errors}")
    else:
        print("Dados inseridos com sucesso no BigQuery.")

# Dados validados para inserção
data_to_insert = [
    {"funcionario_nome": "João Silva", "cpf": "123.456.789-00", "rubrica": "Salário", "valor": 5000.00},
    {"funcionario_nome": "Maria Oliveira", "cpf": "987.654.321-00", "rubrica": "Vale Transporte", "valor": -150.00},
]

# Inserir dados no BigQuery
insert_into_bigquery("auditoria_folha_dataset", "LinhasFolhaFuncionario", data_to_insert)

# Consulta de Status do Job

Nesta seção, será demonstrado como consultar o status de um job de processamento assíncrono utilizando a API do backend.

In [ ]:
import time

# Função para consultar o status do job
def check_job_status(job_id):
    while True:
        response = requests.get(f"{API_BASE_URL}/api/v1/clientes/{CLIENT_ID}/folhas/importar-pdf-async/status/{job_id}")
        if response.status_code == 200:
            status = response.json().get("status")
            print(f"Status do Job: {status}")
            if status in ["CONCLUIDO_SUCESSO", "CONCLUIDO_COM_PENDENCIAS", "FALHA_DOCAI", "FALHA_MAPEAMENTO"]:
                break
        else:
            print(f"Erro ao consultar status: {response.text}")
            break
        time.sleep(5)  # Aguardar 5 segundos antes de consultar novamente

# Exemplo de uso
job_id = "exemplo_job_id"  # Substituir pelo ID real do job
check_job_status(job_id)

# Visualização de Resultados

Nesta seção, será demonstrado como visualizar os resultados processados e armazenados no BigQuery, utilizando gráficos e tabelas interativas.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Simulação de dados processados
data_processed = [
    {"funcionario_nome": "João Silva", "rubrica": "Salário", "valor": 5000.00},
    {"funcionario_nome": "Maria Oliveira", "rubrica": "Vale Transporte", "valor": -150.00},
]

# Criar DataFrame para visualização
df_processed = pd.DataFrame(data_processed)

# Exibir tabela de resultados
print("Resultados Processados:")
print(df_processed)

# Gráfico de barras para valores por funcionário
df_processed.groupby("funcionario_nome")["valor"].sum().plot(kind="bar", title="Valores por Funcionário")
plt.xlabel("Funcionário")
plt.ylabel("Valor (R$)")
plt.show()

# Tratamento de Erros

Nesta seção, será demonstrado como identificar e tratar erros ocorridos durante o processamento, utilizando logs e mensagens de erro detalhadas.

In [ ]:
# Simulação de erros durante o processamento
def process_data_with_error_handling(data):
    for record in data:
        try:
            # Simular processamento
            if record.get("valor") < 0:
                raise ValueError(f"Valor inválido para {record['funcionario_nome']}: {record['valor']}")
            print(f"Processado com sucesso: {record}")
        except ValueError as ve:
            print(f"Erro de valor: {ve}")
        except Exception as e:
            print(f"Erro inesperado: {e}")

# Dados de exemplo com erro
data_with_errors = [
    {"funcionario_nome": "João Silva", "valor": 5000.00},
    {"funcionario_nome": "Maria Oliveira", "valor": -150.00},
]

# Processar dados com tratamento de erros
process_data_with_error_handling(data_with_errors)

In [ ]:
# Configuração do Ambiente
import os
from google.cloud import documentai_v1beta3 as documentai
from google.cloud import storage
from google.cloud import bigquery

# Configurar autenticação do Google Cloud
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'path/to/your-service-account-key.json'

# Variáveis globais
PROJECT_ID = 'seu-projeto-id'
BUCKET_NAME = 'seu-bucket-name'
PROCESSOR_ID = 'seu-processor-id'

In [ ]:
# Configuração do Document AI
from google.cloud import documentai_v1beta3 as documentai
from google.cloud import storage
import os

# Configurações iniciais
PROJECT_ID = "seu-projeto-id"
LOCATION = "us"  # Localização do processador
PROCESSOR_ID = "seu-processor-id"  # ID do processador configurado no Document AI

# Inicializar cliente do Document AI
docai_client = documentai.DocumentProcessorServiceClient()

# Caminho do processador
processor_path = docai_client.processor_path(PROJECT_ID, LOCATION, PROCESSOR_ID)

print(f"Processador configurado: {processor_path}")

In [ ]:
# Função para processar PDF com Document AI
from google.cloud.documentai_v1beta3.types import GcsDocument
from google.cloud.documentai_v1beta3.types import BatchDocumentsInputConfig

def process_pdf_with_docai(gcs_uri):
    # Configurar documento no GCS
    gcs_document = GcsDocument(gcs_uri=gcs_uri, mime_type="application/pdf")
    input_config = BatchDocumentsInputConfig(gcs_documents={"documents": [gcs_document]})

    # Criar requisição
    request = documentai.ProcessRequest(
        name=processor_path,
        raw_document=None,
        input_documents=input_config,
        skip_human_review=True
    )

    # Processar documento
    result = docai_client.process_document(request=request)
    document = result.document

    print("Processamento concluído com sucesso!")
    return document

# Exemplo de uso
gcs_uri_example = "gs://seu-bucket/exemplo.pdf"
document_result = process_pdf_with_docai(gcs_uri_example)